In [ ]:
## Crea una tabla en una BBDD con un campo de caracteres de longitud 25. 
## Inserta valores de longitud mayor a 25 y comprueba

In [54]:
import pandas as pd
### El fichero pokemon_roto.csv está corrupto!
### Observa a Charmander e intenta transformar en entero la columna #
### ¿Puedes arreglarlo?
pokemon = pd.read_csv('pokemon_roto.csv')
pokemon[pokemon['Name'] == 'Charmander']['#'].astype(int) # parece que esto funciona
# pokemon['#'] = pokemon['#'].astype(int) # Nos da error: invalid literal for int() with base 10: 'Ampharos'
pokemon[pokemon['#'] == 'Ampharos'] # Parece que hay un error y pone 'Ampharos' en lugar de un numero y obervamos que es el indice 195
pokemon.iloc[190:200] # Aqui observamos que Ampharos esta entre 180 y 181 asi q le ponemos 180
# pokemon['#'][195] = 180 # salta un warning y nos recomienda remplazar la linea por la siguiente:
pokemon.loc[195, "#"] = '180'
# pokemon['#'] = pokemon['#'].astype(int) # vuelve a dar error por lo que podemos intuir que no es algo muy puntual y no sabemos cuantos puede haber

errores = []
for j, i in enumerate(pokemon['#']): # ¿Bucle for puede llegar a ser muy lento, que otra forma existiria?
    try:
        int(i)
    except:
        pokemon.loc[j, "#"] = pokemon.loc[j - 1, "#"] # sabemos que en la posicion 0 no hay error
        # le estamos dando el valor del anterior # donde esta el error

pokemon['#'] = pokemon['#'].astype(int) # Conseguido


In [67]:
### ¿Puedes localizar aproximadamente el círculo con el mayor entero (25 en esta imagen) sin necesidad de usar ML?
### ¿Puedes analizar si se superpone al círculo azul?

In [ ]:
#### Utiliza la API de Custon Search de Google para encontrar la palabra con más referencias en internet
["Miguel Psellos", "Miguell Sellos", "Miguel Selos"]

In [ ]:
#### Utiliza la libreria PaddleOCR para intentar identificar las palabras en frame_265.jpg
#### Intenta partir la imagen en trozos más pequeños para intentar mejorar los resultados